# Strategy Backtesting with VectorBT

This notebook backtests trading strategies using the MAR indicator framework.

**Two Trading Modes:**
1. **Regime-Only Trading**: Trade based purely on regime detection (no ML)
2. **ML Signal Trading**: Use ML model to find optimal entry/exit points within regimes

**Strategy Logic:**
- **Regime-Only**: Enter long in uptrend, enter short in downtrend, flat in ranging
- **ML Signals**: Use ML predictions for high-confidence entries, exit on regime change or signal flip

**Analysis:**
- Equity curves and drawdowns
- Performance metrics comparison
- Buy-and-hold benchmark
- Parameter sensitivity analysis

## 1. Setup

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import vectorbt as vbt
import joblib
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# Add parent to path
sys.path.insert(0, str(Path('..').resolve()))
from src.config import FEATURES_DIR, MODELS_DIR, LABELS_DIR, INITIAL_CAPITAL, POSITION_SIZE, COMMISSION

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

print(f"VectorBT version: {vbt.__version__}")
print("Setup complete!")

## 2. Configuration

In [ ]:
# Data configuration
SYMBOL = 'BTCUSDT'
INTERVAL = '1h'

# Backtest parameters
INIT_CASH = INITIAL_CAPITAL  # From config
POSITION_PCT = POSITION_SIZE  # From config (0.1 = 10%)
FEES = COMMISSION  # From config (0.001 = 0.1%)

# ML Signal parameters
ML_CONFIDENCE_THRESHOLD = 0.6  # Minimum probability for ML signals
USE_REGIME_FILTER = True  # Only long in uptrend, short in downtrend

print("Backtest Configuration:")
print(f"  Symbol: {SYMBOL}")
print(f"  Interval: {INTERVAL}")
print(f"  Initial Capital: ${INIT_CASH:,}")
print(f"  Position Size: {POSITION_PCT*100}%")
print(f"  Fees: {FEES*100}%")
print(f"  ML Confidence Threshold: {ML_CONFIDENCE_THRESHOLD}")
print(f"  Use Regime Filter: {USE_REGIME_FILTER}")

## 3. Load Data

In [ ]:
# Try to load labeled features (with regime column)
labeled_path = FEATURES_DIR / f'{SYMBOL}_{INTERVAL}_features_labeled.parquet'
regular_path = FEATURES_DIR / f'{SYMBOL}_{INTERVAL}_features.parquet'

if labeled_path.exists():
    print(f"Loading labeled features from: {labeled_path}")
    df = pd.read_parquet(labeled_path)
    has_regime = 'regime' in df.columns
else:
    print(f"Labeled features not found at {labeled_path}")
    print(f"Loading regular features from: {regular_path}")
    print("\nWARNING: Run notebook 02_regime_tuner.ipynb to generate regime labels first!")
    df = pd.read_parquet(regular_path)
    has_regime = False

# Ensure datetime index
df['open_time'] = pd.to_datetime(df['open_time'])
df = df.sort_values('open_time').reset_index(drop=True)

print(f"\nLoaded {len(df):,} bars")
print(f"Date range: {df['open_time'].min()} to {df['open_time'].max()}")
print(f"Price range: ${df['low'].min():,.0f} - ${df['high'].max():,.0f}")
print(f"Has regime labels: {has_regime}")

if has_regime:
    print(f"\nRegime Distribution:")
    regime_map = {0: 'Ranging', 1: 'Trending Up', 2: 'Trending Down'}
    for r, name in regime_map.items():
        count = (df['regime'] == r).sum()
        pct = count / len(df) * 100
        print(f"  {name}: {count:,} ({pct:.1f}%)")

df.head()

## 4. Load ML Model (Optional)

In [ ]:
# Try to load ML signal model
model_path = MODELS_DIR / f'{SYMBOL}_{INTERVAL}_signal_model.joblib'
scaler_path = MODELS_DIR / f'{SYMBOL}_{INTERVAL}_signal_scaler.joblib'
metadata_path = MODELS_DIR / f'{SYMBOL}_{INTERVAL}_signal_metadata.json'

has_ml_model = False
model = None
scaler = None
feature_cols = None

if model_path.exists() and scaler_path.exists():
    print(f"Loading ML model from: {model_path}")
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    
    if metadata_path.exists():
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
            feature_cols = metadata['feature_cols']
            print(f"\nModel Metadata:")
            print(f"  Created: {metadata.get('created_at', 'Unknown')}")
            print(f"  Features: {len(feature_cols)}")
            print(f"  Target params: {metadata.get('target_params', {})}")
    
    has_ml_model = True
    print(f"\nML model loaded successfully!")
else:
    print(f"ML model not found at {model_path}")
    print("\nWARNING: Run notebook 03_signal_ml.ipynb to train ML model first!")
    print("Will only test regime-based strategies.")

print(f"\nHas ML Model: {has_ml_model}")

## 5. Generate ML Predictions (if available)

In [ ]:
if has_ml_model and feature_cols is not None:
    print("Generating ML predictions...")
    
    # Prepare features
    X = df[feature_cols].values
    
    # Handle any NaN values
    valid_mask = ~np.isnan(X).any(axis=1)
    print(f"Valid samples: {valid_mask.sum():,} / {len(df):,}")
    
    # Scale and predict
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    probabilities = model.predict_proba(X_scaled)
    
    # Add to dataframe
    df['ml_signal'] = predictions
    df['ml_prob_no_trade'] = probabilities[:, 0]
    df['ml_prob_long'] = probabilities[:, 1]
    df['ml_prob_short'] = probabilities[:, 2]
    
    # Mark invalid predictions
    df.loc[~valid_mask, 'ml_signal'] = 0
    df.loc[~valid_mask, ['ml_prob_no_trade', 'ml_prob_long', 'ml_prob_short']] = np.nan
    
    print(f"\nML Signal Distribution:")
    signal_map = {0: 'No Trade', 1: 'Long', 2: 'Short'}
    for s, name in signal_map.items():
        count = (df['ml_signal'] == s).sum()
        pct = count / len(df) * 100
        print(f"  {name}: {count:,} ({pct:.1f}%)")
    
    # High-confidence signals
    high_conf_long = (df['ml_prob_long'] > ML_CONFIDENCE_THRESHOLD).sum()
    high_conf_short = (df['ml_prob_short'] > ML_CONFIDENCE_THRESHOLD).sum()
    print(f"\nHigh-Confidence Signals (prob > {ML_CONFIDENCE_THRESHOLD}):")
    print(f"  Long: {high_conf_long:,}")
    print(f"  Short: {high_conf_short:,}")
else:
    print("Skipping ML predictions (model not available)")

## 6. Strategy Definitions

We'll test multiple strategies:
1. **Buy & Hold**: Baseline benchmark
2. **Regime-Only**: Trade based on regime detection
3. **Regime-Filtered**: Only long in uptrend, only short in downtrend
4. **ML Signals**: High-confidence ML signals
5. **ML + Regime**: ML signals filtered by regime

In [ ]:
strategies = {}

# === STRATEGY 1: BUY & HOLD ===
buy_hold_entries = pd.Series(False, index=df.index)
buy_hold_entries.iloc[0] = True
buy_hold_exits = pd.Series(False, index=df.index)

strategies['buy_hold'] = {
    'entries': buy_hold_entries,
    'exits': buy_hold_exits,
    'short_entries': None,
    'short_exits': None,
    'description': 'Buy at start and hold'
}

# === STRATEGY 2: REGIME-ONLY ===
if has_regime:
    # Enter long when entering uptrend, exit when leaving uptrend
    regime_long_entries = (df['regime'] == 1) & (df['regime'].shift(1) != 1)
    regime_long_exits = (df['regime'] != 1) & (df['regime'].shift(1) == 1)
    
    # Enter short when entering downtrend, exit when leaving downtrend
    regime_short_entries = (df['regime'] == 2) & (df['regime'].shift(1) != 2)
    regime_short_exits = (df['regime'] != 2) & (df['regime'].shift(1) == 2)
    
    strategies['regime_only'] = {
        'entries': regime_long_entries,
        'exits': regime_long_exits,
        'short_entries': regime_short_entries,
        'short_exits': regime_short_exits,
        'description': 'Trade on regime changes (long in uptrend, short in downtrend)'
    }
    
    # === STRATEGY 3: REGIME LONG-ONLY ===
    strategies['regime_long_only'] = {
        'entries': regime_long_entries,
        'exits': regime_long_exits,
        'short_entries': None,
        'short_exits': None,
        'description': 'Long only during uptrends'
    }

# === STRATEGY 4: ML SIGNALS (if available) ===
if has_ml_model:
    # High-confidence long signals
    ml_long_entries = df['ml_prob_long'] > ML_CONFIDENCE_THRESHOLD
    ml_long_exits = df['ml_prob_long'] <= ML_CONFIDENCE_THRESHOLD
    
    # High-confidence short signals
    ml_short_entries = df['ml_prob_short'] > ML_CONFIDENCE_THRESHOLD
    ml_short_exits = df['ml_prob_short'] <= ML_CONFIDENCE_THRESHOLD
    
    strategies['ml_signals'] = {
        'entries': ml_long_entries,
        'exits': ml_long_exits,
        'short_entries': ml_short_entries,
        'short_exits': ml_short_exits,
        'description': f'ML signals (confidence > {ML_CONFIDENCE_THRESHOLD})'
    }
    
    # === STRATEGY 5: ML + REGIME FILTER ===
    if has_regime:
        # Only long in uptrend, only short in downtrend
        ml_regime_long_entries = ml_long_entries & (df['regime'] == 1)
        ml_regime_long_exits = ml_long_exits | (df['regime'] != 1)
        
        ml_regime_short_entries = ml_short_entries & (df['regime'] == 2)
        ml_regime_short_exits = ml_short_exits | (df['regime'] != 2)
        
        strategies['ml_regime_filtered'] = {
            'entries': ml_regime_long_entries,
            'exits': ml_regime_long_exits,
            'short_entries': ml_regime_short_entries,
            'short_exits': ml_regime_short_exits,
            'description': 'ML signals filtered by regime (long in uptrend, short in downtrend)'
        }

print(f"Defined {len(strategies)} strategies:")
for name, strat in strategies.items():
    print(f"  - {name}: {strat['description']}")

## 7. Run Backtests

In [ ]:
print("Running backtests...\n")
print(f"Parameters: Initial Cash=${INIT_CASH:,}, Position Size={POSITION_PCT*100}%, Fees={FEES*100}%")
print("="*80)

portfolios = {}

for name, strat in strategies.items():
    print(f"\nRunning: {name}")
    print(f"  {strat['description']}")
    
    try:
        # Determine position size (100% for buy & hold, else configured %)
        size = 1.0 if name == 'buy_hold' else POSITION_PCT
        
        # Run backtest
        pf = vbt.Portfolio.from_signals(
            close=df['close'],
            entries=strat['entries'],
            exits=strat['exits'],
            short_entries=strat['short_entries'],
            short_exits=strat['short_exits'],
            init_cash=INIT_CASH,
            size=size,
            size_type='percent',
            fees=FEES,
            freq=f'{INTERVAL}'
        )
        
        portfolios[name] = pf
        
        # Quick stats
        print(f"  Total Trades: {pf.trades.count()}")
        print(f"  Final Value: ${pf.final_value():,.2f}")
        print(f"  Total Return: {pf.total_return()*100:.2f}%")
        
    except Exception as e:
        print(f"  ERROR: {e}")

print("\n" + "="*80)
print(f"Successfully ran {len(portfolios)} backtests!")

## 8. Performance Metrics Comparison

In [ ]:
# Collect metrics
metrics_data = []

for name, pf in portfolios.items():
    metrics_data.append({
        'Strategy': name.replace('_', ' ').title(),
        'Total Return (%)': pf.total_return() * 100,
        'CAGR (%)': pf.annualized_return() * 100,
        'Sharpe Ratio': pf.sharpe_ratio(),
        'Sortino Ratio': pf.sortino_ratio(),
        'Max Drawdown (%)': pf.max_drawdown() * 100,
        'Win Rate (%)': pf.trades.win_rate() * 100 if pf.trades.count() > 0 else 0,
        'Profit Factor': pf.trades.profit_factor() if pf.trades.count() > 0 else 0,
        'Total Trades': pf.trades.count(),
        'Final Value ($)': pf.final_value()
    })

metrics_df = pd.DataFrame(metrics_data)
metrics_df = metrics_df.sort_values('Total Return (%)', ascending=False)

print("\nPERFORMANCE METRICS COMPARISON")
print("="*120)
print(metrics_df.to_string(index=False))

# Save to CSV
output_path = Path('..') / 'logs' / f'backtest_results_{SYMBOL}_{INTERVAL}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
output_path.parent.mkdir(parents=True, exist_ok=True)
metrics_df.to_csv(output_path, index=False)
print(f"\nResults saved to: {output_path}")

## 9. Equity Curves

In [ ]:
# Plot all equity curves together
fig = go.Figure()

colors = {'buy_hold': 'orange', 'regime_only': 'blue', 'regime_long_only': 'cyan',
          'ml_signals': 'green', 'ml_regime_filtered': 'purple'}

for name, pf in portfolios.items():
    display_name = name.replace('_', ' ').title()
    color = colors.get(name, 'gray')
    
    fig.add_trace(go.Scatter(
        x=df['open_time'],
        y=pf.value(),
        mode='lines',
        name=display_name,
        line=dict(width=2, color=color)
    ))

fig.add_hline(y=INIT_CASH, line_dash='dash', line_color='white', 
              annotation_text='Initial Capital', opacity=0.5)

fig.update_layout(
    template='plotly_dark',
    title='Strategy Equity Curves Comparison',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    height=600,
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

## 10. Drawdown Analysis

In [ ]:
# Plot drawdowns for each strategy
n_strategies = len(portfolios)
fig = make_subplots(
    rows=n_strategies, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=[name.replace('_', ' ').title() for name in portfolios.keys()]
)

for i, (name, pf) in enumerate(portfolios.items(), 1):
    dd = pf.drawdown() * 100
    color = colors.get(name, 'gray')
    
    fig.add_trace(go.Scatter(
        x=df['open_time'],
        y=dd,
        mode='lines',
        fill='tozeroy',
        name=name.replace('_', ' ').title(),
        line=dict(color=color, width=1),
        fillcolor=color.replace(')', ', 0.3)').replace('rgb', 'rgba') if 'rgb' in color else f'rgba(255, 0, 0, 0.3)'
    ), row=i, col=1)
    
    fig.update_yaxes(title_text='DD (%)', row=i, col=1)

fig.update_layout(
    template='plotly_dark',
    title='Drawdown Comparison',
    height=200 * n_strategies,
    showlegend=False
)

fig.update_xaxes(title_text='Date', row=n_strategies, col=1)

fig.show()

## 11. Metrics Visualization

In [ ]:
# Create bar charts for key metrics
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Strategy Performance Metrics', fontsize=16, fontweight='bold')

# Total Return
axes[0, 0].barh(metrics_df['Strategy'], metrics_df['Total Return (%)'])
axes[0, 0].set_xlabel('Return (%)')
axes[0, 0].set_title('Total Return')
axes[0, 0].grid(True, alpha=0.3)

# Sharpe Ratio
axes[0, 1].barh(metrics_df['Strategy'], metrics_df['Sharpe Ratio'])
axes[0, 1].set_xlabel('Sharpe Ratio')
axes[0, 1].set_title('Sharpe Ratio')
axes[0, 1].axvline(x=0, color='black', linestyle='--', alpha=0.3)
axes[0, 1].grid(True, alpha=0.3)

# Max Drawdown
axes[0, 2].barh(metrics_df['Strategy'], -metrics_df['Max Drawdown (%)'], color='red')
axes[0, 2].set_xlabel('Drawdown (%)')
axes[0, 2].set_title('Max Drawdown')
axes[0, 2].grid(True, alpha=0.3)

# Win Rate
axes[1, 0].barh(metrics_df['Strategy'], metrics_df['Win Rate (%)'])
axes[1, 0].set_xlabel('Win Rate (%)')
axes[1, 0].set_title('Win Rate')
axes[1, 0].axvline(x=50, color='black', linestyle='--', alpha=0.3, label='50% baseline')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Profit Factor
axes[1, 1].barh(metrics_df['Strategy'], metrics_df['Profit Factor'])
axes[1, 1].set_xlabel('Profit Factor')
axes[1, 1].set_title('Profit Factor')
axes[1, 1].axvline(x=1, color='black', linestyle='--', alpha=0.3, label='Break-even')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Total Trades
axes[1, 2].barh(metrics_df['Strategy'], metrics_df['Total Trades'])
axes[1, 2].set_xlabel('Number of Trades')
axes[1, 2].set_title('Total Trades')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Trade Analysis (Best Strategy)

In [ ]:
# Find best performing strategy (by Sharpe ratio)
best_strategy_name = metrics_df.iloc[0]['Strategy'].lower().replace(' ', '_')
best_pf = portfolios[best_strategy_name]

print(f"Analyzing: {best_strategy_name.replace('_', ' ').title()}")
print("="*60)

# Trade statistics
if best_pf.trades.count() > 0:
    print("\nTRADE STATISTICS")
    print(best_pf.trades.stats())
    
    # Get trade returns
    trade_returns = best_pf.trades.returns.values * 100
    
    # Plot distribution
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(trade_returns, bins=50, alpha=0.7, edgecolor='black')
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
    axes[0].axvline(x=trade_returns.mean(), color='green', linestyle='--', linewidth=2, 
                    label=f'Mean: {trade_returns.mean():.2f}%')
    axes[0].set_xlabel('Return (%)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Trade Return Distribution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Cumulative returns
    cumulative_returns = np.cumsum(trade_returns)
    axes[1].plot(cumulative_returns, linewidth=2)
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Trade Number')
    axes[1].set_ylabel('Cumulative Return (%)')
    axes[1].set_title('Cumulative Trade Returns')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Win/Loss breakdown
    wins = trade_returns[trade_returns > 0]
    losses = trade_returns[trade_returns < 0]
    
    print(f"\nWIN/LOSS BREAKDOWN")
    print(f"  Total Trades: {len(trade_returns)}")
    print(f"  Winning Trades: {len(wins)} ({len(wins)/len(trade_returns)*100:.1f}%)")
    print(f"  Losing Trades: {len(losses)} ({len(losses)/len(trade_returns)*100:.1f}%)")
    print(f"  Average Win: {wins.mean():.2f}%" if len(wins) > 0 else "  Average Win: N/A")
    print(f"  Average Loss: {losses.mean():.2f}%" if len(losses) > 0 else "  Average Loss: N/A")
    print(f"  Largest Win: {trade_returns.max():.2f}%")
    print(f"  Largest Loss: {trade_returns.min():.2f}%")
    print(f"  Win/Loss Ratio: {abs(wins.mean() / losses.mean()):.2f}" if len(wins) > 0 and len(losses) > 0 else "  Win/Loss Ratio: N/A")
else:
    print("No trades executed.")

## 13. Parameter Sensitivity Analysis

Test how performance varies with different position sizes and confidence thresholds.

In [ ]:
# Position Size Sensitivity
print("Testing position size sensitivity...")
position_sizes = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.50, 0.75, 1.00]

# Use best non-buy-hold strategy
test_strategies = [name for name in portfolios.keys() if name != 'buy_hold']
if len(test_strategies) > 0:
    test_strat_name = test_strategies[0]
    test_strat = strategies[test_strat_name]
    
    position_results = []
    
    for pos_size in position_sizes:
        pf = vbt.Portfolio.from_signals(
            close=df['close'],
            entries=test_strat['entries'],
            exits=test_strat['exits'],
            short_entries=test_strat['short_entries'],
            short_exits=test_strat['short_exits'],
            init_cash=INIT_CASH,
            size=pos_size,
            size_type='percent',
            fees=FEES,
            freq=f'{INTERVAL}'
        )
        
        position_results.append({
            'Position Size (%)': pos_size * 100,
            'Total Return (%)': pf.total_return() * 100,
            'Sharpe Ratio': pf.sharpe_ratio(),
            'Max Drawdown (%)': pf.max_drawdown() * 100,
            'Final Value ($)': pf.final_value()
        })
    
    position_df = pd.DataFrame(position_results)
    
    print(f"\nPosition Size Sensitivity ({test_strat_name}):")
    print(position_df.to_string(index=False))
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Return vs Position Size
    axes[0].plot(position_df['Position Size (%)'], position_df['Total Return (%)'], 
                 marker='o', linewidth=2, markersize=8)
    axes[0].axvline(x=POSITION_PCT*100, color='red', linestyle='--', alpha=0.5, 
                    label=f'Current ({POSITION_PCT*100}%)')
    axes[0].set_xlabel('Position Size (%)')
    axes[0].set_ylabel('Total Return (%)')
    axes[0].set_title('Total Return vs Position Size')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Sharpe vs Position Size
    axes[1].plot(position_df['Position Size (%)'], position_df['Sharpe Ratio'], 
                 marker='o', linewidth=2, markersize=8, color='green')
    axes[1].axvline(x=POSITION_PCT*100, color='red', linestyle='--', alpha=0.5, 
                    label=f'Current ({POSITION_PCT*100}%)')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[1].set_xlabel('Position Size (%)')
    axes[1].set_ylabel('Sharpe Ratio')
    axes[1].set_title('Sharpe Ratio vs Position Size')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No strategies available for sensitivity analysis.")

## 14. ML Confidence Threshold Sensitivity (if ML available)

In [ ]:
if has_ml_model:
    print("Testing ML confidence threshold sensitivity...")
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    threshold_results = []
    
    for threshold in thresholds:
        # Generate signals with this threshold
        long_entries = df['ml_prob_long'] > threshold
        long_exits = df['ml_prob_long'] <= threshold
        short_entries = df['ml_prob_short'] > threshold
        short_exits = df['ml_prob_short'] <= threshold
        
        # Optionally filter by regime
        if has_regime and USE_REGIME_FILTER:
            long_entries = long_entries & (df['regime'] == 1)
            long_exits = long_exits | (df['regime'] != 1)
            short_entries = short_entries & (df['regime'] == 2)
            short_exits = short_exits | (df['regime'] != 2)
        
        pf = vbt.Portfolio.from_signals(
            close=df['close'],
            entries=long_entries,
            exits=long_exits,
            short_entries=short_entries,
            short_exits=short_exits,
            init_cash=INIT_CASH,
            size=POSITION_PCT,
            size_type='percent',
            fees=FEES,
            freq=f'{INTERVAL}'
        )
        
        threshold_results.append({
            'Threshold': threshold,
            'Total Return (%)': pf.total_return() * 100,
            'Sharpe Ratio': pf.sharpe_ratio(),
            'Win Rate (%)': pf.trades.win_rate() * 100 if pf.trades.count() > 0 else 0,
            'Total Trades': pf.trades.count(),
            'Final Value ($)': pf.final_value()
        })
    
    threshold_df = pd.DataFrame(threshold_results)
    
    print(f"\nML Confidence Threshold Sensitivity:")
    print(threshold_df.to_string(index=False))
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Return vs Threshold
    axes[0].plot(threshold_df['Threshold'], threshold_df['Total Return (%)'], 
                 marker='o', linewidth=2, markersize=8)
    axes[0].axvline(x=ML_CONFIDENCE_THRESHOLD, color='red', linestyle='--', alpha=0.5, 
                    label=f'Current ({ML_CONFIDENCE_THRESHOLD})')
    axes[0].set_xlabel('Confidence Threshold')
    axes[0].set_ylabel('Total Return (%)')
    axes[0].set_title('Return vs Confidence Threshold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Sharpe vs Threshold
    axes[1].plot(threshold_df['Threshold'], threshold_df['Sharpe Ratio'], 
                 marker='o', linewidth=2, markersize=8, color='green')
    axes[1].axvline(x=ML_CONFIDENCE_THRESHOLD, color='red', linestyle='--', alpha=0.5, 
                    label=f'Current ({ML_CONFIDENCE_THRESHOLD})')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
    axes[1].set_xlabel('Confidence Threshold')
    axes[1].set_ylabel('Sharpe Ratio')
    axes[1].set_title('Sharpe Ratio vs Confidence Threshold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Trade Count vs Threshold
    axes[2].plot(threshold_df['Threshold'], threshold_df['Total Trades'], 
                 marker='o', linewidth=2, markersize=8, color='orange')
    axes[2].axvline(x=ML_CONFIDENCE_THRESHOLD, color='red', linestyle='--', alpha=0.5, 
                    label=f'Current ({ML_CONFIDENCE_THRESHOLD})')
    axes[2].set_xlabel('Confidence Threshold')
    axes[2].set_ylabel('Total Trades')
    axes[2].set_title('Trade Count vs Confidence Threshold')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("ML model not available. Skipping threshold sensitivity analysis.")

## 15. Visualize Signals on Price Chart

In [ ]:
# Plot recent price action with signals from best strategy
n_display = 500
df_plot = df.tail(n_display).copy()

# Get best strategy signals
best_strat = strategies[best_strategy_name]

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=(f'{best_strategy_name.replace("_", " ").title()} - Recent Signals', 'Portfolio Value')
)

# Add regime backgrounds if available
if has_regime:
    regime_colors = {
        0: 'rgba(255, 255, 0, 0.1)',   # Yellow - Ranging
        1: 'rgba(0, 255, 0, 0.1)',     # Green - Trending Up
        2: 'rgba(255, 0, 0, 0.1)'      # Red - Trending Down
    }
    
    current_regime = None
    block_start = None
    
    for idx in range(len(df_plot)):
        regime = df_plot.iloc[idx]['regime']
        if regime != current_regime:
            if current_regime is not None and block_start is not None:
                fig.add_vrect(
                    x0=df_plot.iloc[block_start]['open_time'],
                    x1=df_plot.iloc[idx-1]['open_time'],
                    fillcolor=regime_colors[current_regime],
                    layer="below", line_width=0,
                    row=1, col=1
                )
            current_regime = regime
            block_start = idx
    
    # Close final block
    if current_regime is not None:
        fig.add_vrect(
            x0=df_plot.iloc[block_start]['open_time'],
            x1=df_plot.iloc[-1]['open_time'],
            fillcolor=regime_colors[current_regime],
            layer="below", line_width=0,
            row=1, col=1
        )

# Candlestick
fig.add_trace(go.Candlestick(
    x=df_plot['open_time'],
    open=df_plot['open'], high=df_plot['high'],
    low=df_plot['low'], close=df_plot['close'],
    name='Price'
), row=1, col=1)

# Entry signals
entries_plot = df_plot[best_strat['entries'].iloc[-n_display:]]
if len(entries_plot) > 0:
    fig.add_trace(go.Scatter(
        x=entries_plot['open_time'],
        y=entries_plot['low'] * 0.998,
        mode='markers',
        marker=dict(symbol='triangle-up', size=12, color='green'),
        name='Long Entry'
    ), row=1, col=1)

# Short entry signals (if applicable)
if best_strat['short_entries'] is not None:
    short_entries_plot = df_plot[best_strat['short_entries'].iloc[-n_display:]]
    if len(short_entries_plot) > 0:
        fig.add_trace(go.Scatter(
            x=short_entries_plot['open_time'],
            y=short_entries_plot['high'] * 1.002,
            mode='markers',
            marker=dict(symbol='triangle-down', size=12, color='red'),
            name='Short Entry'
        ), row=1, col=1)

# Portfolio value
pf_values = best_pf.value().iloc[-n_display:]
fig.add_trace(go.Scatter(
    x=df_plot['open_time'],
    y=pf_values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='cyan', width=2)
), row=2, col=1)

fig.add_hline(y=INIT_CASH, line_dash='dash', line_color='white', opacity=0.5, row=2, col=1)

fig.update_layout(
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False,
    hovermode='x unified'
)

fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='Value ($)', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)

fig.show()

## 16. Summary Statistics

In [ ]:
print("="*80)
print("BACKTEST SUMMARY")
print("="*80)

print(f"\nData: {SYMBOL} {INTERVAL}")
print(f"Period: {df['open_time'].min()} to {df['open_time'].max()}")
print(f"Total Bars: {len(df):,}")

print(f"\nBacktest Configuration:")
print(f"  Initial Capital: ${INIT_CASH:,}")
print(f"  Position Size: {POSITION_PCT*100}%")
print(f"  Commission: {FEES*100}%")

print(f"\nBest Strategy: {best_strategy_name.replace('_', ' ').title()}")
best_metrics = metrics_df.iloc[0]
print(f"  Total Return: {best_metrics['Total Return (%)']:.2f}%")
print(f"  CAGR: {best_metrics['CAGR (%)']:.2f}%")
print(f"  Sharpe Ratio: {best_metrics['Sharpe Ratio']:.2f}")
print(f"  Max Drawdown: {best_metrics['Max Drawdown (%)']:.2f}%")
print(f"  Win Rate: {best_metrics['Win Rate (%)']:.2f}%")
print(f"  Total Trades: {int(best_metrics['Total Trades'])}")
print(f"  Final Value: ${best_metrics['Final Value ($)']:,.2f}")

# Compare to buy & hold
if 'buy_hold' in portfolios:
    bh_metrics = metrics_df[metrics_df['Strategy'] == 'Buy Hold'].iloc[0]
    print(f"\nVs Buy & Hold:")
    print(f"  Return Difference: {best_metrics['Total Return (%)'] - bh_metrics['Total Return (%)']:.2f}%")
    print(f"  Sharpe Improvement: {best_metrics['Sharpe Ratio'] - bh_metrics['Sharpe Ratio']:.2f}")
    print(f"  Drawdown Improvement: {bh_metrics['Max Drawdown (%)'] - best_metrics['Max Drawdown (%)']:.2f}%")

print("\n" + "="*80)

---

## Next Steps

Based on the backtest results, consider:

1. **If regime-only performs well**: The MAR indicator regime detection is effective. Focus on refining regime parameters in notebook 02.

2. **If ML signals perform well**: The ML model is adding value. Consider:
   - Tuning the confidence threshold
   - Training with different target parameters (notebook 03)
   - Adding more features

3. **If ML + regime filter performs best**: The combination is powerful. This validates the two-stage approach.

4. **For deployment**:
   - Implement proper risk management (stop-loss, take-profit)
   - Add position sizing based on volatility
   - Consider walk-forward optimization
   - Test on out-of-sample data
   - Paper trade before live trading

5. **Further analysis**:
   - Monte Carlo simulation
   - Different timeframes
   - Multiple symbols
   - Regime-specific parameters